<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/cycle_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CycleGAN
## 目的
Cycle GANのネットワークを構築して動作を理解する．

# データセット
りんごとオレンジが含まれるデータセットのzipファイルをダウンロードして，解凍します．
もし違うデータを使いたい場合は，以下のURLから選択してURL先のデータをwgetで取得してください．<br>
https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/

In [ ]:
!wget -q https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip -O apple2orange.zip
!unzip -q -o apple2orange.zip

# 各種モジュールのインポート

In [ ]:
import os
import random
import itertools
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ネットワークの構築
CycleGANは，「画像を入力→異なるスタイルへ変換→変換したものを元のスタイルへ戻す」という処理によって，対になる画像がない場合であってもスタイルを変換できる代物です．<br>
以下に示す図は，CycleGANによる変換及び再構成を表しています．


<img src="https://dl.dropboxusercontent.com/s/7339ohcadohs6o7/Cyclegan.png" width=40%>



CycleGANが提案される前は，pix2pixと呼ばれるペア画像を必要とするスタイル変換が主流でした．
ペア画像とは，例えばRGB画像とその画像に対するセグメンテーション画像のことを指します．<br>

CycleGANのGeneratorはResidual networkをベースに設計します．
Residual network (ResNet)は，残差機構を用いることで多層になった場合でも特徴を残すことが可能なネットワークです．
ResBlockは，カーネルサイズ3×3，ストライド1，パディング1の2層の畳み込み層で構築します．
以下にResBlockの図を示します．

<img src="https://dl.dropboxusercontent.com/s/pow1wsxhc37gmts/ResBlock.png" width=50%>

ここで$f(x)$は，前層が出力した特徴マップを表しています．
また$g(x)$は，残差を加算した特徴マップです．


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, features):
        super(ResBlock, self).__init__()
        block_list = []
        self.block = self.make_block(block_list, features)
        
    def make_block(self, modules_list, features):
        modules_list.append(nn.ReflectionPad2d(1))
        modules_list.append(nn.Conv2d(features, features, kernel_size=3, stride=1, bias=True))
        modules_list.append(self.select_normalization(norm='instance', features=features))
        modules_list.append(nn.ReLU(inplace=True))
        modules_list.append(nn.ReflectionPad2d(1))
        modules_list.append(nn.Conv2d(features, features, kernel_size=3, stride=1, bias=True))
        modules_list.append(self.select_normalization(norm='instance', features=features))
        modules = nn.Sequential(*modules_list)
        return modules
        
    def select_normalization(self, norm, features):
        if norm == 'batch':
            return nn.BatchNorm2d(features)
        elif norm == 'instance':
            return nn.InstanceNorm2d(features)
        else:
            assert 0, '%s is not supported.' % norm

    def forward(self, x):
        out = x + self.block(x)
        return out

CycleGANのGeneratorは，入力に潜在変数ではなく変換元となる画像を入力して，画像を出力します．そのため，GeneratorはEncoder-Decoder構造をしています．Encoderは，入力画像を表現するために有益な情報を残した特徴抽出をします．一方で，DecoderはEncoderがエンコードした特徴を用いて画像の変換をします．このときにEncoderとDecoderの間には，任意の数積み上げたResBlockを使用することで，さらに特徴を凝縮します．
ここで，ResBlockは，残差を用いてネットワークを構築するResidual Networksで使用されているものです．

<img src="https://dl.dropboxusercontent.com/s/yha9im0bbb6p0t1/CycleGAN_G.png" width=50%>

In [ ]:
class Generator(nn.Module):
    def __init__(self, n_down, n_up, n_res, in_features):
        super(Generator, self).__init__()
        
        out_features = 64
        first_conv = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_features, out_features, kernel_size=7, stride=1, padding=0, bias=True),
            self.select_normalization(norm='instance', features=out_features),
            nn.ReLU(inplace=True)]
        
        down_block = []
        for _ in range(n_down):
            in_features = out_features
            out_features = in_features * 2
            down_block += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=True),
                self.select_normalization(norm='instance', features=out_features),
                nn.ReLU(inplace=True)]
            
        res_block = []
        res_features = out_features
        for _ in range(n_res):
            res_block.append(ResBlock(res_features))
            
        up_block = []
        in_features = res_features
        out_features = in_features // 2
        for _ in range(n_up):
            up_block += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1, bias=True),
                self.select_normalization(norm='instance', features=out_features),
                nn.ReLU(inplace=True)]
            in_features = out_features
            out_features = in_features // 2
        
        last_conv = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_features, 3, kernel_size=7, stride=1, padding=0, bias=True),
            nn.Tanh()]
        
        self.first_conv = nn.Sequential(*first_conv)
        self.down_block = nn.Sequential(*down_block)
        self.res_block = nn.Sequential(*res_block)
        self.up_block = nn.Sequential(*up_block)
        self.last_conv = nn.Sequential(*last_conv)
        self.init_weights(self.first_conv)
        self.init_weights(self.down_block)
        self.init_weights(self.res_block)
        self.init_weights(self.up_block)
        self.init_weights(self.last_conv)

    def init_weights(self, net):
        classname = net.__class__.__name__
        if classname.find('Conv') != -1:
            torch.nn.init.normal_(net.weight.data, 0.0, 0.02)
            if hasattr(net, 'bias') and net.bias is not None:
                torch.nn.init.constant_(net.bias.data, 0.0)
    
    def select_normalization(self, norm, features):
        if norm == 'batch':
            return nn.BatchNorm2d(features)
        elif norm == 'instance':
            return nn.InstanceNorm2d(features)
        else:
            assert 0, '%s is not supported.' % norm
            
    def forward(self, x):
        h = self.first_conv(x)
        h = self.down_block(h)
        h = self.res_block(h)
        h = self.up_block(h)
        out = self.last_conv(h)   
        return out

Discriminatorは，実画像であるかGeneratorによって変換された画像であるかの分類をするため，通常のGANと同じ振る舞いです．
DC-GANは，画像1枚を入力して実画像であるか生成画像であるかを判定しています．
つまり，Dicriminatorは画像全体を見て判断していると言っても良いでしょう．<br>
ところが，CycleGANはPatchGANをベースにDiscriminatorを構築しているので，少し変わった方法で実画像か生成画像かの判定をしています．
PatchGANは，画像全体ではなく画像をいくつかのパッチに区切って入力することで，局所領域ごとに実画像なのか生成画像なのかを判定します．
これによって，画像全体で判定するよりも性能がよくなることが知られています．
以下に通常のDiscriminatorとCycleGANのDsicriminatorを示します．



<img src="https://dl.dropboxusercontent.com/s/yrud8444g26g0yz/CycleGAN_D.png" width=50%>

図にも示したように，厳密には任意の数のPatchに分割することは手間なので，Discriminator内部の畳み込み処理のカーネルサイズによって受容野の広さを制御して，出力値をスカラー値ではなく特徴マップとすることによってPatchに分割したときと同様の処理を実現しています．

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, n_layers=3):
        super(Discriminator, self).__init__()
        out_features = 64
        modules = [nn.Conv2d(3, out_features, kernel_size=4, stride=2, padding=1, bias=True),
                   nn.LeakyReLU(negative_slope=0.2, inplace=True)]

        for i in range(n_layers):
            in_features = out_features
            out_features = in_features * 2
            if i == n_layers - 1:    stride=1
            else:    stride=2
            modules += [nn.Conv2d(in_features, out_features, kernel_size=4, stride=stride, padding=1, bias=True),
                        self.select_normalization(norm='instance', features=out_features),
                        nn.LeakyReLU(negative_slope=0.2, inplace=True)]
        
        modules += [nn.Conv2d(out_features, 1, kernel_size=4, stride=1, padding=1, bias=True)]
        self.layers = nn.Sequential(*modules)
        self.init_weights(self.layers)

    def init_weights(self, net):
        classname = net.__class__.__name__
        if classname.find('Conv') != -1:
            torch.nn.init.normal_(net.weight.data, 0.0, 0.02)
            if hasattr(net, 'bias') and net.bias is not None:
                torch.nn.init.constant_(net.bias.data, 0.0)
    
    def select_normalization(self, norm, features):
        if norm == 'batch':
            return nn.BatchNorm2d(features)
        elif norm == 'instance':
            return nn.InstanceNorm2d(features)
        else:
            assert 0, '%s is not supported.' % norm
            
    def forward(self, x):
        out = self.layers(x)
        return out

# DataLoaderの定義
pytorchに含まれているデータ以外のデータセットを用いて学習するときや特殊なデータ構造を利用して学習をする時は，DataLoaderを自分で定義する必要があります．今回使用するデータセットは，zipファイルを解凍すると，内部にtrainA，trainB，testA，testBという名前のディレクトリが含まれています．
また，それぞれのディレクトリの中に画像が入っています．

これらを考慮して以下に示すようにDataLoaderを定義します．

datapath：zipファイルを解凍したときにできるディレクトリのパス


In [ ]:
class CycleGAN_Dataset(torch.utils.data.Dataset):
    def __init__(self, datapath, transforms=None):
        self.transforms = transforms
        self.A_path = os.path.join(datapath, 'trainA')
        self.B_path = os.path.join(datapath, 'trainB')
        dataA_list = os.listdir(self.A_path)
        dataB_list = os.listdir(self.B_path)
        random.shuffle(dataA_list)
        random.shuffle(dataB_list)
        self.datalength = min(len(dataA_list), len(dataB_list))
        self.dataA = dataA_list[:self.datalength]
        self.dataB = dataB_list[:self.datalength]
        
    def __len__(self):
        return self.datalength
    
    def __getitem__(self, i):
        imgA = Image.open(os.path.join(self.A_path, self.dataA[i]))
        imgB = Image.open(os.path.join(self.B_path, self.dataB[i]))
        
        if self.transforms:
            imgA = self.transforms(imgA)
            imgB = self.transforms(imgB)
        
        return imgA, imgB

# Image History Buffer
CycleGANでは，任意のサイズのBufferを定義します．
これにより，過去にDiscriminatorをうまく騙せた画像に依存して変換のバリエーションが少なくなる問題を回避することができます．

Image Bufferは，Generatorが変換した画像を溜め込みますが，任意のiteration数で中身の画像を最新の画像にアップデートします．このときに，全ての画像を破棄して新しいものへアップデートするのではなく，一部のみをアップデートします．つまり，異なるiteration数で変換した画像が混在したBufferとなっています．

In [ ]:
class Image_History_Buffer:
    def __init__(self, pool_size=50):
        self.pool_size = pool_size
        self.buffer = []
    
    def get_images(self,pre_images):
        return_imgs = []
        for img in pre_images:
            img = torch.unsqueeze(img,0)
            if len(self.buffer) < self.pool_size:
                self.buffer.append(img)
                return_imgs.append(img)
            else:
                if random.randint(0,1)>0.5:
                    i = random.randint(0,self.pool_size-1)
                    tmp = self.buffer[i].clone()
                    self.buffer[i]=img
                    return_imgs.append(tmp)
                else:
                    return_imgs.append(img)
        return torch.cat(return_imgs,dim=0)

In [ ]:
class loss_scheduler():
    def __init__(self, epoch_decay):
        self.epoch_decay = epoch_decay

    def f(self, epoch):
        if epoch<=self.epoch_decay:
            return 1
        else:
            scaling = 1 - (epoch-self.epoch_decay)/float(self.epoch_decay)
            return scaling

# ネットワークの作成，学習に必要なパラメータの定義
CycleGANは，ドメインAからBに変換するGeneratorとその逆の処理をするGeneratorの2つ作成する必要があります．また，Discriminatorも同様で，ドメインAのためのDiscriminator，ドメインBのためのDicriminatorも作成します．

In [ ]:
lr = 0.0002
img_size = 256
betas = (0.5, 0.999)
batchsize = 1
imgsize = 256
n_epochs = 200
decay_epoch = 100
lambda_val = 10
lambda_id_val = 0
datapath = 'apple2orange'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Make training dataset
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
transform = transforms.Compose([transforms.Resize(img_size, Image.BICUBIC),
                                   transforms.RandomCrop(imgsize),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean, std)])
train_data = CycleGAN_Dataset(datapath=datapath, transforms=transform)
training_dataset = DataLoader(train_data, batch_size=batchsize, shuffle=True)

# Define networks
G_A2B = Generator(n_down=2, n_up=2, n_res=9, in_features=3).to(device)
G_B2A = Generator(n_down=2, n_up=2, n_res=9, in_features=3).to(device)
D_A = Discriminator(n_layers=3).to(device)
D_B = Discriminator(n_layers=3).to(device)

g_opt = optim.Adam(itertools.chain(G_A2B.parameters(), G_B2A.parameters()), lr=lr, betas=betas)
d_A_opt = optim.Adam(D_A.parameters(), lr=lr, betas=betas)
d_B_opt = optim.Adam(D_B.parameters(), lr=lr, betas=betas)

g_lr_scheduler = torch.optim.lr_scheduler.LambdaLR(g_opt, lr_lambda=loss_scheduler(decay_epoch).f)
d_a_lr_scheduler = torch.optim.lr_scheduler.LambdaLR(d_A_opt, lr_lambda=loss_scheduler(decay_epoch).f)
d_b_lr_scheduler = torch.optim.lr_scheduler.LambdaLR(d_B_opt, lr_lambda=loss_scheduler(decay_epoch).f)

adv_loss = nn.MSELoss()
l1_norm = nn.L1Loss()
criterion_idn = nn.L1Loss()

buffer_for_fakeA = Image_History_Buffer()
buffer_for_fakeB = Image_History_Buffer()

# CycleGANの学習
これまでに定義したネットワークを用いて学習します．

CycleGANの誤差関数は，オリジナルのGANとは異なるものが使用されています．
* Real/Fakeの誤差関数：GANを学習するために必要な誤差関数です．LS-GANで提案されているカイ2乗誤差を誤差関数とする方法を利用しています．
$$\mathcal{L}_{adv}^{A} = \mathbb{E}\left[\left(D_{A}(x_{A}) - 1\right)^{2}\right] + \mathbb{E}\left[D_{A}\left(G_{B\rightarrow A}(x_{B})\right)^{2}\right]$$
$$\mathcal{L}_{adv}^{B} = \mathbb{E}\left[\left(D_{B}(x_{B}) - 1\right)^{2}\right] + \mathbb{E}\left[D_{B}\left(G_{A\rightarrow B}(x_{A})\right)^{2}\right]$$
* Cycle consistency loss：オリジナルの画像と再構成した画像の一貫性を保つために必要な誤差関数です．
$$
\mathcal{L}_{cycle}^{A} = \|x_{A} - G_{B\rightarrow A}\left(G_{A\rightarrow B}\left(x_{A}\right)\right)\|_{1}
$$
$$
\mathcal{L}_{cycle}^{B} = \|x_{B} - G_{A\rightarrow B}\left(G_{B\rightarrow A}\left(x_{B}\right)\right)\|_{1}
$$
* Identity loss：別のスタイルへ変換したときにオリジナル画像を大幅に変更してしまうことを抑制するために必要な誤差です．実際，この誤差を抜いても学習は可能で，スタイルの変換もすることはできます．この誤差関数が最大の効果を発揮する場面は，ある風景画を絵画風に変換するときです．
$$
\mathcal{L}_{identity}^{A} = \|x_{A} - G_{A\rightarrow B}\left(x_{A}\right)\|_{1}
$$
$$
\mathcal{L}_{identity}^{B} = \|x_{B} - G_{B\rightarrow A}\left(x_{B}\right)\|_{1}
$$

In [ ]:
for epoch in range(1, n_epochs+1):
    G_B2A.train()
    G_A2B.train()
    D_A.train()
    D_B.train()
    for idx, (imgA, imgB) in enumerate(training_dataset):
        imgA = Variable(imgA.to(device), requires_grad=True)
        imgB = Variable(imgB.to(device), requires_grad=True)
        imgA_fake, imgB_fake = G_B2A(imgB), G_A2B(imgA)
        imgA_rec, imgB_rec = G_B2A(imgB_fake), G_A2B(imgA_fake)
        if lambda_id_val > 0:
            iden_imgA, iden_imgB = G_B2A(imgA), G_A2B(imgB)
        
        # Update the discriminator (D_A, D_B)
        d_A_opt.zero_grad()
        disA_out_real = D_A(imgA)
        imgA_fake_ = buffer_for_fakeA.get_images(imgA_fake)
        disA_out_fake = D_A(imgA_fake_.detach())
        d_lossA_real = adv_loss(disA_out_real, torch.tensor(1.0).expand_as(disA_out_real).to(device))
        d_lossA_fake = adv_loss(disA_out_fake, torch.tensor(0.0).expand_as(disA_out_fake).to(device))
        disA_loss = (d_lossA_real + d_lossA_fake) * 0.5
        disA_loss.backward()
        d_A_opt.step()
        
        d_B_opt.zero_grad()
        disB_out_real = D_B(imgB)
        imgB_fake_ = buffer_for_fakeB.get_images(imgB_fake)
        disB_out_fake = D_B(imgB_fake_.detach())
        d_lossB_real = adv_loss(disB_out_real, torch.tensor(1.0).expand_as(disB_out_real).to(device))
        d_lossB_fake = adv_loss(disB_out_fake, torch.tensor(0.0).expand_as(disA_out_fake).to(device))
        disB_loss = (d_lossB_real + d_lossB_fake) * 0.5
        disB_loss.backward()
        d_B_opt.step()
        
        # Update the generator (G)
        g_opt.zero_grad()
        disB_out_fake = D_B(imgB_fake)
        disA_out_fake = D_A(imgA_fake)
        g_lossA = adv_loss(disA_out_fake, torch.tensor(1.0).expand_as(disA_out_fake).to(device))
        g_lossB = adv_loss(disB_out_fake, torch.tensor(1.0).expand_as(disB_out_fake).to(device))
        gen_adv_loss = g_lossA + g_lossB
        
        cycle_consistency_loss = l1_norm(imgA_rec, imgA) + l1_norm(imgB_rec, imgB)
        if lambda_id_val > 0:
            identity_loss = criterion_idn(iden_imgA, imgA) + criterion_idn(iden_imgB, imgB)
            gen_loss = gen_adv_loss + lambda_val * cycle_consistency_loss + lambda_id_val * identity_loss
        else:
            gen_loss = gen_adv_loss + lambda_val * cycle_consistency_loss
        gen_loss.backward()
        g_opt.step()
        
        if idx % 100 == 0:
            print('Training epoch: {} [{}/{} ({:.0f}%)] | D loss (A): {:.6f} | D loss (B): {:.6f} | G loss: {:.6f} | Consistency: {:.6f} | Identity: {:.6f} |'\
                  .format(epoch, idx * len(imgA), len(training_dataset.dataset),
                  100. * idx / len(training_dataset), disA_loss.item(), disB_loss.item(), gen_loss.item(), cycle_consistency_loss.item(), identity_loss.item()))

# 学習済みモデルのロード
CycleGANはネットワークが深いため，綺麗な変換をするためには長時間の学習を必要とします．
Colabratory上で学習できなくはないですが，手早くCycleGANが出力する結果を見たい方は，以下を動作させて学習済みモデルをダウンロードしてください．

ディレクトリ内には，appleとorangeの相互変換をするモデルとhorseとzebraの相互変換するモデルが含まれています．

In [ ]:
!wget -q https://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/CycleGAN_pretrained_model.zip --no-check-certificate
!unzip -q -o CycleGAN_pretrained_model.zip

In [ ]:
data_name = 'apple2orange'
G_A2B_path = os.path.join('./CycleGAN_pretrained_model', data_name, 'G_A2B')
G_B2A_path = os.path.join('./CycleGAN_pretrained_model', data_name, 'G_B2A')
D_A_path = os.path.join('./CycleGAN_pretrained_model', data_name, 'D_A')
D_B_path = os.path.join('./CycleGAN_pretrained_model', data_name, 'D_B')
G_A2B.load_state_dict(torch.load(G_A2B_path))
G_B2A.load_state_dict(torch.load(G_B2A_path))
D_A.load_state_dict(torch.load(D_A_path))
D_B.load_state_dict(torch.load(D_B_path))

In [ ]:
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
normalize = transforms.Normalize(mean=mean, std=std)
to_tensor = transforms.ToTensor()
transform = transforms.Compose([to_tensor, normalize])

domainA_path = os.path.join(data_name, 'testA')
domainB_path = os.path.join(data_name, 'testB')
imgA_list, imgB_list = os.listdir(domainA_path), os.listdir(domainB_path)
indexA, indexB = np.random.randint(len(imgA_list)), np.random.randint(len(imgB_list))
imgA = Image.open(os.path.join(domainA_path, imgA_list[indexA]))
imgB = Image.open(os.path.join(domainB_path, imgB_list[indexB]))
imgA_tensor = transform(imgA).to(device)[None,:,:,:]
imgB_tensor = transform(imgB).to(device)[None,:,:,:]
G_A2B.eval()
G_B2A.eval()
with torch.no_grad():
    fake_B = G_A2B(imgA_tensor)
    fake_A = G_B2A(imgB_tensor)
    rec_B = G_A2B(fake_A)
    rec_A = G_B2A(fake_B)
    mean = torch.tensor(mean, dtype=torch.float32)[None,:,None,None].to(device)
    std = torch.tensor(std, dtype=torch.float32)[None,:,None,None].to(device)
    fake_B = (fake_B * std) + mean
    fake_A = (fake_A * std) + mean

fake_imgA = Image.fromarray((fake_A * 256.).clamp(min=0, max=255).data.cpu().squeeze().permute(1,2,0).numpy().astype(np.uint8))
fake_imgB = Image.fromarray((fake_B * 256.).clamp(min=0, max=255).data.cpu().squeeze().permute(1,2,0).numpy().astype(np.uint8))
plt_items = [imgA, fake_imgB, imgB, fake_imgA]
title_list = ['Real_A', 'Fake_B', 'Real_B', 'Fake_A']
rows = 2
cols = 2
axes=[]
fig=plt.figure(figsize=(16, 9))

for i in range(rows*cols):
    item = plt_items[i]
    axes.append( fig.add_subplot(rows, cols, i+1) )
    axes[-1].set_title(title_list[i])
    plt.axis('off')

    plt.imshow(item)
fig.tight_layout()    
plt.show()

In [ ]:
rec_A = (rec_A * std) + mean
Image.fromarray((rec_A * 256.).clamp(min=0, max=255).data.cpu().squeeze().permute(1,2,0).numpy().astype(np.uint8))

# 課題


*   データセットを変更して学習してみましょう．
*   変換するデータと学習済みモデルが一致していない場合の画像変換をしてみましょう



# 参考文献
[1] Phillip Isola, Jun-Yan Zhu, Tinghui Zhou and Alexei A. Efros, Image-to-Image Translation with Conditional Adversarial Nets, CVPR, 2017.\
[2] Jun-Yan Zhu, Taesung Park, Phillip Isola and Alexei A. Efros, Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks, ICCV, 2017.